<a href="https://colab.research.google.com/github/Mwadz/Path_description/blob/main/New_data_Description_Geneneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Loading the Data

In [5]:
import pandas as pd
# reading the data and naming it desc
desc = pd.read_csv('/content/description_train_data.csv')
desc.head(2)
print(desc.shape)

(1250, 2)


In [6]:
raw = pd.read_csv('/content/document_data_for_training_description - Copy.csv')
raw.head(2)
print(raw.shape)

(3747, 2)


# Cleaning and Preparing Data

In [7]:
#@title Renaming the columns 
desc.columns = ['input_text', 'target_text']
raw.columns = ['input_text', 'target_text']
# Dropping the target column in the "raw" dataset so we use it on testing
raw = raw.drop(['target_text'], axis = 1)
#taking out the first part
raw = raw.applymap(lambda x: str(x).split(':')[-1])

In [8]:
#@title Dealing with special characters
import warnings
warnings.filterwarnings('ignore')
desc['input_text'] = desc['input_text'].str.replace('{', '')
desc['target_text'] = desc['target_text'].str.replace('}', '')
desc['target_text'] = desc['target_text'].str.replace('"', '')
raw['input_text'] = raw['input_text'].str.replace('}', '')
raw['input_text'] = raw['input_text'].str.replace('', '')
desc.head(2)
#desc['input_text'] = desc['input_text'].str.replace('/', ' ')
# desc['input_text'] = desc['input_text'].str.replace('_', ' ')


# creating a primary key
#desc['row_id'] = range(1, len(desc.index)+1)

,input_text,target_text
0,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,A free-form text field used to describe the s...
1,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Indicates whether the comparable property has...


In [9]:
# checking for nulls
desc.isnull().sum()

input_text     0
target_text    1
dtype: int64

In [10]:
# Dropping the null values
desc = desc[desc['target_text'].notna()]
desc.isnull().sum()

input_text     0
target_text    0
dtype: int64

In [11]:
# checking for duplicate values
desc.duplicated().sum()

54

In [12]:
desc.drop_duplicates(keep=False, inplace=False)

,input_text,target_text
41,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,The date on which a community received its fi...
53,/DOCUMENT/REFERENCE/LocationURL/@DataNotSuppli...,A value from a MISMO prescribed list that spe...
54,/DOCUMENT/REFERENCE/LocationURL/@DataNotSuppli...,A free-form text field used to provide a supp...
55,/DOCUMENT/REFERENCE/LocationURL/@DataNotSuppli...,A free-form text field used to capture the en...
56,/DOCUMENT/REFERENCE/LocationURL/@SensitiveIndi...,Allows producers and consumers of XML instanc...
...,...,...
1245,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,The month and year the next property casualty...
1246,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Additional Description of the Hazard Insuranc...
1247,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,A free-form text field used to capture the Ha...
1248,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Information about one or more types of car st...


In [13]:
desc.duplicated().sum()


54

In [14]:
# sampling the data so test is a bit less than train
desc_eval = desc.sample(frac=0.1) 
print(desc_eval.shape)
# desc_train = desc.copy()
# desc_train.shape

(125, 2)


In [15]:
desc = desc.sample(frac=0.2) 
print(desc.shape)

(250, 2)


In [16]:
desc_eval.head()

,input_text,target_text
1149,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,Specifies the multiplier factor used in calcu...
1058,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,The number of individual family dwelling unit...
438,/DOCUMENT/DEAL_SETS/DEAL_SET/ACH/ACHDraftFrequ...,A value from a MISMO prescribed list that spe...
366,/DOCUMENT/AUDIT_TRAIL/AUDIT_TRAIL_ENTRIES/AUDI...,Allows producers and consumers of XML instanc...
1106,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,The date corresponding to the status of the p...


In [17]:
desc.head()

,input_text,target_text
646,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,A free-form text field used to capture the en...
1087,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,The identifier of the locked box located in t...
705,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,A free-form text field used to provide a supp...
420,/DOCUMENT/DEAL_SETS/DEAL_SET/ACH/ACHBankAccoun...,A free-form text field used to provide a supp...
795,/DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS...,The date on which the value represented by th...


<font color = orange>**Sidenote**</font>
* In standard practice train data usually makes up 70-80% of the total data while test data takes the rest ie 20-30%
* This should be taken into consideration while evaluating performance

# Modeling

## i) Using Bart (Baseline Model)


In [18]:
import pandas as pd
import gensim
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

In [19]:
# setting hyperparameters
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 5
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True

# Initializing the model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
    use_cuda= False,
)

In [20]:
def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum(
        [
            1 if label == pred else 0
            for label, pred in zip(labels, preds)
        ]
    ) 

In [21]:
# Training the model
model.train_model(desc, eval_data=desc_eval)

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/16 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/32 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/125 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/16 [00:00<?, ?it/s]

(160,
 {'global_step': [32, 64, 96, 128, 160],
  'eval_loss': [8.242081463336945,
   3.8308906257152557,
   2.441600024700165,
   1.5494150519371033,
   1.3130427151918411],
  'train_loss': [8.987677574157715,
   4.278071403503418,
   3.046595811843872,
   2.1839663982391357,
   1.5041598081588745]})

In [22]:
# # Evaluating the model
results = model.eval_model(desc_eval)
results


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/125 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 1.3130427151918411}

In [23]:
# Use the model for prediction
print(model.predict(
        [
            "DOCUMENT/SIGNATORIES/SIGNATORY/EXECUTION/EXECUTION_DETAIL/ActualSignatureTypeOtherDescription"
        ]
    )
)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[' A free-form text field used to capture the enumeration when Other is']
